In [1]:
class Ticket:
    def __init__(self, id:int):
        self.id = id

    def __str__(self):
        return f"ticket id: {self.id}"

In [2]:
class TicketMachine:
    def __init__(self):
        self.tickets = []
        self.index = 0
    
    def __str__(self):
        return f"tickets: {[ticket.__str__() for ticket in self.tickets]}"
    
    def getNewTicket(self):
        new_ticket = Ticket(len(self.tickets))
        self.tickets.append(new_ticket)
        return new_ticket
    
    def nextTicket(self):
        self.index += 1
        return self.getCurrentTicket()
    
    def getCurrentTicket(self):
        if self.index < len(self.tickets):   
            return self.tickets[self.index]
        else:
            return None

In [3]:
gep = TicketMachine()

In [4]:
sorszam = gep.getNewTicket()
print(sorszam)
print(f"A gepben levo sorszamok: {gep}")

ticket id: 0
A gepben levo sorszamok: tickets: ['ticket id: 0']


In [5]:
lista = [1, 2,  3]

[i*i for i in lista] # Ezt amugy hivjak list comprehension-nek a pythonosok

[1, 4, 9]

In [6]:
class Customer:
    def buyTicket(self, machine:TicketMachine):
        self.ticket = machine.getNewTicket()
        
    def showTicket(self):
        return self.ticket
    

In [7]:
bela_ba = Customer()
bela_ba.buyTicket(gep)
print(bela_ba.ticket)

jozsi_ba = Customer()
jozsi_ba.buyTicket(gep)
print(jozsi_ba.ticket)

ticket id: 1
ticket id: 2


In [8]:
class Staff:
    def __init__(self, machine:TicketMachine):
        self.machine = machine
        self.busy = False
        self.currentTicket = None

    def validateTicket(self, ticket:Ticket):
        if ticket == self.currentTicket:
            return True
        return False
    
    def shoutoutTicket(self):
        if self.currentTicket is None:
            self.currentTicket = self.machine.getCurrentTicket()
            self.machine.nextTicket()
        return self.currentTicket
    
    def invalidateTicket(self):
        self.currentTicket is None
        self.busy = False
        
    def dealwithTicket(self, ticket: Ticket):
        self.busy = self.validateTicket(ticket)
        return self.busy
        

In [9]:
a = None
if a is None:
    print("a is none")
if a == None:
    print("a == none")

a is none
a == none


In [10]:
jolika = Staff(gep)
gizike = Staff(gep)


In [11]:
print(jolika.shoutoutTicket())
print(jolika.validateTicket(bela_ba.showTicket()))
print(jolika.validateTicket(jozsi_ba.showTicket()))
print(jolika.validateTicket(sorszam))

print(gizike.shoutoutTicket())
print(gizike.validateTicket(bela_ba.showTicket()))
print(gizike.validateTicket(jozsi_ba.showTicket()))
print(gizike.validateTicket(sorszam))

ticket id: 0
False
False
True
ticket id: 1
True
False
False


In [12]:
jolika.invalidateTicket()

In [13]:
#Test01:
#shop opens and machine starts. 
sorszamgep = TicketMachine()
print(sorszamgep)
#staff members are ready at the cashier
ibolya = Staff(sorszamgep)
peter = Staff(sorszamgep)

#customers arrived
robi_ba = Customer()
erzsi = Customer()

#customers request numbers
robi_ba.buyTicket(sorszamgep)
print(f"Robi bá sorszáma:{robi_ba.showTicket()}")
erzsi.buyTicket(sorszamgep)
print(f"Erzsi sorszáma:{erzsi.showTicket()}")

#staff members ask for new customers
print(f"Ibolya is waiting for: {ibolya.shoutoutTicket()}")
print(f"Peter is waiting for: {peter.shoutoutTicket()}")

#staff members deal with customers
print(f"Erzsi jön Ibolyához: {ibolya.dealwithTicket(erzsi.showTicket())}")
print(f"Robi bá jön Peterhez: {peter.dealwithTicket(robi_ba.showTicket())}")
print(f"Robi bá jön Ibolyához: {ibolya.dealwithTicket(robi_ba.showTicket())}")
print(f"Erzsi jön Peterhez: {peter.dealwithTicket(erzsi.showTicket())}")


tickets: []
Robi bá sorszáma:ticket id: 0
Erzsi sorszáma:ticket id: 1
Ibolya is waiting for: ticket id: 0
Peter is waiting for: ticket id: 1
Erzsi jön Ibolyához: False
Robi bá jön Peterhez: False
Robi bá jön Ibolyához: True
Erzsi jön Peterhez: True
